# **Imports and Libraries**

In [1]:
!pip install transformers datasets scikit-learn pandas numpy
!pip install --upgrade transformers

# Standard libraries
import os
import random

# Data manipulation libraries
import numpy as np
import pandas as pd

# PyTorch related
import torch
from torch.optim.swa_utils import AveragedModel
from safetensors.torch import load_file

# Hugging Face transformers
from transformers import (
    RobertaConfig,
    RobertaForSequenceClassification,
    RobertaTokenizer,
    Trainer,
    TrainerCallback,
    TrainingArguments
)
from datasets import Dataset

# Google Colab
from google.colab import drive

# Set Constant Seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(42)

# CSV Display
from IPython.display import display

# Mount Drive
drive.mount('/content/drive/')
save_path = '/content/drive/MyDrive/nlu'
os.makedirs(save_path, exist_ok=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

# **Generate Prediction from One Pair**

In [2]:
# Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Model
save_path = "/content/drive/MyDrive/nlu/swa_final_model_last"
config = RobertaConfig.from_pretrained(save_path)
model = RobertaForSequenceClassification(config)
state_dict = load_file(os.path.join(save_path, "model.safetensors"))
model.load_state_dict(state_dict)
model.to(device)
model.eval()
eval_trainer = Trainer(model=model)
print("Model loaded from safetensors successfully")

def tokenize(batch):
    return tokenizer(batch["premise"], batch["hypothesis"],
                     truncation=True, padding="max_length", max_length=128)

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained(save_path)

# Define a custom sentence pair
premise = "A boy in a red shirt is riding his bicycle down the street."
hypothesis = "The boy is playing outside."

# Tokenize the custom sentence pair
inputs = tokenizer(premise, hypothesis,
                   truncation=True, padding="max_length", max_length=128,
                   return_tensors="pt")
inputs = {key: val.to(device) for key, val in inputs.items()}

# Run inference
with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits

# Get prediction
prediction = torch.argmax(logits, dim=1).item()

print("")
print("Premise: ", premise)
print("Hypothesis: ", hypothesis)
print("Prediction:", prediction)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Model loaded from safetensors successfully

Premise:  A boy in a red shirt is riding his bicycle down the street.
Hypothesis:  The boy is playing outside.
Prediction: 1


# **Generate Predictions for Test Data**

In [12]:
# Load the test data
dataset_df = pd.read_csv('/content/drive/MyDrive/nlu/test.csv')

# Tokenize
dataset_ds = Dataset.from_pandas(dataset_df).map(tokenize, batched=True)
dataset_ds.set_format(type="torch", columns=["input_ids", "attention_mask"])

# Run inference
pred_output = eval_trainer.predict(dataset_ds)
pred_logits = pred_output.predictions
pred_labels = np.argmax(pred_logits, axis=1)

# Save only the prediction column
pd.DataFrame({"prediction": pred_labels}).to_csv("Group_15_C.csv", index=False)

print("Saved Group_15_C.csv with", len(pred_labels), "rows.")
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Load and display the predictions CSV
predictions_df = pd.read_csv("Group_15_C.csv")
display(predictions_df)

Map:   0%|          | 0/3302 [00:00<?, ? examples/s]

Saved Group_15_C.csv with 3302 rows.


,prediction
0,1
1,1
2,1
3,1
4,1
5,0
6,0
7,0
8,0
9,1
